In [ ]:
import torch
import argparse
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import importlib
from importlib import reload

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# If you are using Colab
dir_path = "/content/drive/Othercomputers/my_computer/dl-nlp_project_named-entity-recognition/"
module_path = dir_path[9:].replace("/", ".")
# imports
data_module = importlib.import_module(module_path + "data_new")
prepare_data_pipeline = data_module.prepare_data_pipeline
TRAIN_DATA_PATH = data_module.TRAIN_DATA_PATH
TEST_DATA_PATH = data_module.TEST_DATA_PATH
PAD = data_module.PAD
tensor_to_sentences = data_module.tensor_to_sentences
tensor_to_labels = data_module.tensor_to_labels

In [ ]:
# If you are NOT using colab
# dir_path = ""
# from data_new import (
#     prepare_data_pipeline,
#     TRAIN_DATA_PATH,
#     TEST_DATA_PATH,
#     PAD,
#     tensor_to_sentences,
#     tensor_to_labels,
# )

In [ ]:
train_file_path = dir_path + "data/train.json"
test_file_path = dir_path + "data/test.json"

In [ ]:
reload(data_module)

In [ ]:
class SimpleRNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super(SimpleRNNModel, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # Simple RNN layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        # Convert token indices to embeddings
        embedded = self.embedding(text)

        # Pass embeddings through RNN
        rnn_output, _ = self.rnn(embedded)

        # Pass RNN output through dense layer
        predictions = self.fc(rnn_output)

        return predictions

In [ ]:
def train(model, train_loader, optimizer, criterion, device, PAD_IDX):
    """
    Training logic for an epoch
    """
    model.train()

    epoch_loss = 0

    for batch in train_loader:
        sentences = batch["sentence"]
        labels = batch["label"]
        sentences, labels = sentences.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        predictions = model(sentences)
        mask = (sentences != PAD_IDX).float()
        mask = mask.unsqueeze(-1).expand_as(labels)

        # Compute loss (takes logits as input)
        loss = criterion(
            predictions,
            labels
        )
        loss = (loss * mask).mean() # mask out 'PAD' tokens

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

In [ ]:
def validate(model, val_loader, criterion, device):
    """
    Validation logic with micro-F1 score
    """
    model.eval()

    epoch_loss = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            sentences = batch["sentence"]
            labels = batch["label"]
            sentences, labels = sentences.to(device), labels.to(device)

            predictions = model(sentences)
            mask = (sentences != PAD_IDX).float()
            mask = mask.unsqueeze(-1).expand_as(labels)

            # Convert sigmoid outputs to binary labels
            binary_predictions = (torch.sigmoid(predictions) >= 0.5).float()

            all_predictions.append(binary_predictions.view(-1).cpu().numpy())
            all_labels.append(labels.view(-1).cpu().numpy())

            # this loss is not really necessary
            loss = criterion(
                predictions,
                labels
            )
            loss = (loss * mask).mean() # mask out 'PAD' tokens
            epoch_loss += loss.item()

    # Compute micro-F1 score
    micro_f1 = f1_score(
        np.hstack(all_labels), np.hstack(all_predictions), average="micro"
    )

    return epoch_loss / len(val_loader), micro_f1

In [ ]:
def evaluate(model, test_loader, device, idx_to_word, idx_to_label):
    """
    Evaluation logic with micro-F1 score
    """
    model.eval()

    epoch_loss = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            sentences = batch["sentence"]
            labels = batch["label"]
            sentences, labels = sentences.to(device), labels.to(device)

            predictions = model(sentences)

            # Convert sigmoid outputs to binary labels
            binary_predictions = (torch.sigmoid(predictions) >= 0.5).float()

            # uncomment this if you want to see the predictions
            # sigmoid_predictions = torch.sigmoid(predictions)

            # print(sentences.shape) # 32, 181
            # decoded_sentences = tensor_to_sentences(sentences, idx_to_word)

            # print(labels.shape) # 32, 181, 36
            # decoded_labels = tensor_to_labels(labels, idx_to_label)

            # print(predictions.shape) # 32, 181, 36
            # decoded_predictions = tensor_to_labels(binary_predictions, idx_to_label)

            # for i in range(len(decoded_sentences)):
            #     sentence = decoded_sentences[i]
            #     for j in range(len(sentence)):
            #         decoded_word = sentence[j]
            #         encoded_word = sentences[i][j]
            #         decoded_label = decoded_labels[i][j]
            #         encoded_label = labels[i][j].cpu().numpy().tolist()
            #         decoded_prediction = decoded_predictions[i][j]
            #         encoded_prediction = sigmoid_predictions[i][j].cpu().numpy().tolist()
            #         if decoded_word != PAD:
            #             print(decoded_word, decoded_label, decoded_prediction)
            #             print(f"Encoded Word: {encoded_word.item()}")
            #             print(f"Encoded Label:      {encoded_label}")
            #             print(f"Encoded Prediction: {[round(x, 2) for x in encoded_prediction]}")
            #     print()
            # foo()

            all_predictions.append(binary_predictions.view(-1).cpu().numpy())
            all_labels.append(labels.view(-1).cpu().numpy())

    # Compute micro-F1 score
    micro_f1 = f1_score(
        np.hstack(all_labels), np.hstack(all_predictions), average="micro"
    )

    return epoch_loss / len(test_loader), micro_f1

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
(
    train_loader,
    val_loader,
    test_loader,
    MAX_LENGTH,
    word_to_idx,
    idx_to_word,
    label_to_idx,
    idx_to_label,
) = prepare_data_pipeline(train_file_path, test_file_path)

In [ ]:
config = {}
VOCAB_SIZE = len(word_to_idx)
config["embedding_dim"] = 100
config["hidden_dim"] = 128
config["epochs"] = 100
config["lr"] = 0.001
OUTPUT_DIM = len(label_to_idx)  # Number of labels
PAD_IDX = word_to_idx[PAD]

model = SimpleRNNModel(
    VOCAB_SIZE, config["embedding_dim"], config["hidden_dim"], OUTPUT_DIM, PAD_IDX
)
model = model.to(device)
criterion = nn.BCEWithLogitsLoss(reduction='none')
optimizer = optim.Adam(model.parameters(), config["lr"])

for epoch in range(config["epochs"]):
    train_loss = train(
        model, train_loader, optimizer, criterion, device, PAD_IDX
    )
    val_loss, micro_f1 = validate(model, val_loader, criterion, device)

    print(f"Epoch: {epoch+1:02}")
    print(f"\tTrain Loss: {train_loss:.3f}")
    print(f"\t Val. Loss: {val_loss:.3f}")
    print(f"\t Micro-F1 Score (Val): {micro_f1:.3f}")

test_micro_f1 = evaluate(model, test_loader, device, idx_to_word, idx_to_label)
print(f"Micro-F1 Score (Test): {test_micro_f1:.3f}")

In [ ]:
evaluate(model, val_loader, device, idx_to_word, idx_to_label)